In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from modules import torch_classes
from torchvision import transforms, utils
from modules.grad_cam import *

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
#Train set
labels_path = '../DATA/train_labels.pkl'
data_path = 'train.dat'
data_dims = (8269,10404)
genes_path = 'train.csv'

#Data and labels
train = torch_classes.TumorDataset(labels_path,data_path,data_dims,genes_path,transform = transforms.Compose([torch_classes.ToImage(),torch_classes.ToTensor()]))

#Test set
labels_path = '../DATA/test_labels.pkl'
data_path = 'test.dat'
data_dims = (2085,10404)
genes_path = 'test.csv'

test = torch_classes.TumorDataset(labels_path,data_path,data_dims,genes_path,transform = transforms.Compose([torch_classes.ToImage(),torch_classes.ToTensor()]))

In [28]:
num_epochs = 30
batch_size = 75
learning_rate = 0.0001

### Using sklearn stratified kfold splitter, iterate over the folds to run tests

In [29]:
skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=True)
skf.split(np.zeros(len(train.int_labels)), train.int_labels)

<generator object _BaseKFold.split at 0x7f1622e5eb88>

### CV to find optimal number of epochs

In [30]:
training_losses = []
validation_losses = []
fold = 0

for train_index, val_index in skf.split(np.zeros(len(train.int_labels)), train.int_labels):
    train_subset = torch.utils.data.Subset(train, train_index)
    val_subset = torch.utils.data.Subset(train, val_index)
    
    #initialize data loaders using subsets
    dataloader_train = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    dataloader_val = DataLoader(val_subset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    #Reinitialize network for every instance
    net = torch_classes.Net(num_of_classes=33)
    net.to(device)
    net.double()
    
    #Training loss function
    criterion = nn.CrossEntropyLoss()

    #Training optimizer
    optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate)
    
    fold_trn_loss = []
    fold_val_loss = []
    
    for epoch in tqdm(range(num_epochs)):
        running_loss = 0.0
        running_val_loss = 0.0
        
        for i, samples in enumerate(dataloader_train):
            #Data
            images = samples['data'].to(device).requires_grad_()
            images = torch.unsqueeze(images,1)
            labels = samples['label'].to(device)
            _, labels = torch.max(labels,1) #converts from one hot to integer
            
            #Forward Pass
            outputs = net(images)

            #Backward Pass
            optimizer.zero_grad()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.data
            print("Epoch {} Batch {} loss is: {}".format(epoch, i, loss.data))
            
        epoch_training_loss = running_loss.item()/(i+1) #i + 1 since index starts from 0
        
        for j, val_samples in enumerate(dataloader_val):
            #validation loss
            images = val_samples['data'].to(device)
            images = torch.unsqueeze(images,1)
            labels = val_samples['label'].to(device)
            _, labels = torch.max(labels,1)

            #Forward Pass
            outputs = net(images)
            loss = criterion(outputs, labels)
            running_val_loss += loss.data
            
        epoch_validation_loss = running_val_loss.item()/(j+1)
        
        fold_trn_loss.append(epoch_training_loss)
        fold_val_loss.append(epoch_validation_loss)
        print('fold', fold, 'epoch', epoch, 'trn loss is:', epoch_training_loss, 'val loss is:', epoch_validation_loss)
    
    plt.plot(fold_trn_loss, label="Train")
    plt.plot(fold_val_loss, label="Validation")
    plt.title("Training vs Validation Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss (Cross Entropy)')
    plt.legend()
    plt.show()
    
    training_losses.append(fold_trn_loss)
    validation_losses.append(fold_val_loss)
    fold = fold + 1



  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0 Batch 0 loss is: 3.526394989167268
Epoch 0 Batch 1 loss is: 3.734350669301763
Epoch 0 Batch 2 loss is: 3.058990553101805
Epoch 0 Batch 3 loss is: 2.8045508597411852
Epoch 0 Batch 4 loss is: 2.6530753433712113
Epoch 0 Batch 5 loss is: 2.591426679875444
Epoch 0 Batch 6 loss is: 2.9702986891943866
Epoch 0 Batch 7 loss is: 2.0337783129479874
Epoch 0 Batch 8 loss is: 2.2438971160183843
Epoch 0 Batch 9 loss is: 2.282969901815195
Epoch 0 Batch 10 loss is: 2.1202171558270915
Epoch 0 Batch 11 loss is: 2.2085308789001052
Epoch 0 Batch 12 loss is: 2.037771703134282
Epoch 0 Batch 13 loss is: 1.9373709842477929
Epoch 0 Batch 14 loss is: 1.7592555547695437
Epoch 0 Batch 15 loss is: 1.5762485790674152
Epoch 0 Batch 16 loss is: 1.5301700636922906
Epoch 0 Batch 17 loss is: 1.6216274740300602
Epoch 0 Batch 18 loss is: 1.5379127275408697
Epoch 0 Batch 19 loss is: 1.4458990303518962
Epoch 0 Batch 20 loss is: 1.4225203783424403
Epoch 0 Batch 21 loss is: 1.4084797070876958
Epoch 0 Batch 22 loss is: 



  3%|▎         | 1/30 [04:39<2:14:53, 279.07s/it]

fold 0 epoch 0 trn loss is: 1.1094013271466967 val loss is: 0.3838830445016632
Epoch 1 Batch 0 loss is: 0.2340885143330947
Epoch 1 Batch 1 loss is: 0.2772651324737858
Epoch 1 Batch 2 loss is: 0.34755187386783004
Epoch 1 Batch 3 loss is: 0.20636125881134404
Epoch 1 Batch 4 loss is: 0.31229896083467046
Epoch 1 Batch 5 loss is: 0.21393649523737898
Epoch 1 Batch 6 loss is: 0.22157254930137532
Epoch 1 Batch 7 loss is: 0.3229338597527511
Epoch 1 Batch 8 loss is: 0.17659077036290202
Epoch 1 Batch 9 loss is: 0.1902776763903071
Epoch 1 Batch 10 loss is: 0.14315736164162374
Epoch 1 Batch 11 loss is: 0.3185886003954748
Epoch 1 Batch 12 loss is: 0.20752832166916194
Epoch 1 Batch 13 loss is: 0.16161590679751886
Epoch 1 Batch 14 loss is: 0.22207666287020839
Epoch 1 Batch 15 loss is: 0.30388714737887457
Epoch 1 Batch 16 loss is: 0.21956523114472565
Epoch 1 Batch 17 loss is: 0.2825451141188534
Epoch 1 Batch 18 loss is: 0.12416731852666267
Epoch 1 Batch 19 loss is: 0.19800135202662666
Epoch 1 Batch 20 



  7%|▋         | 2/30 [09:16<2:09:58, 278.52s/it]

...



 33%|███▎      | 10/30 [46:04<1:31:47, 275.36s/it]

fold 0 epoch 9 trn loss is: 0.0015491716547504717 val loss is: 0.21318182467005364
Epoch 10 Batch 0 loss is: 0.0014120930086278113
Epoch 10 Batch 1 loss is: 0.0008785628241533535
Epoch 10 Batch 2 loss is: 0.0012272200514882078
Epoch 10 Batch 3 loss is: 0.0018977565882937596
Epoch 10 Batch 4 loss is: 0.0012400651739195468
Epoch 10 Batch 5 loss is: 0.0017705731868221145
Epoch 10 Batch 6 loss is: 0.0012690060671549474
Epoch 10 Batch 7 loss is: 0.0008668498660822622
Epoch 10 Batch 8 loss is: 0.0008914964763643951
Epoch 10 Batch 9 loss is: 0.0014010753146805636
Epoch 10 Batch 10 loss is: 0.0014048225156313284
Epoch 10 Batch 11 loss is: 0.000717700283430555
Epoch 10 Batch 12 loss is: 0.0013951875664357516
Epoch 10 Batch 13 loss is: 0.0017780766079000425
Epoch 10 Batch 14 loss is: 0.0014738156166020624
Epoch 10 Batch 15 loss is: 0.0010321173591292402
Epoch 10 Batch 16 loss is: 0.002049751250878889
Epoch 10 Batch 17 loss is: 0.002008101868946284
Epoch 10 Batch 18 loss is: 0.0015016730771040443



 37%|███▋      | 11/30 [50:39<1:27:12, 275.38s/it]

fold 0 epoch 10 trn loss is: 0.001303215594162098 val loss is: 0.21152194913135589


OSError: [Errno 12] Cannot allocate memory

In [31]:
#This is list of lists of lists
losses = [training_losses,validation_losses]
with open('CV_file/losses/losses.pkl', 'wb') as f:
    pickle.dump(losses, f)

In [ ]:
fig = plt.figure()
for i in range(len(training_losses)):
    plt.subplot(len(training_losses)+1,1,i+1)
    plt.plot(training_losses[i], label="Train")
    plt.plot(validation_losses[i], label="Validation")
    plt.title("Training vs Validation Loss")
    plt.xlabel('Epochs')
    plt.ylabel('Loss (Cross Entropy)')
    plt.legend()
plt.show()
fig.set_size_inches(8, 20)    
fig.savefig("CV_file/Loss.png")